### Objective
* We don't have block group and neighborhood mapping, so this is an attempt to do that by overlaying block group layer and communities/neighborhood layer.

In [5]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection, FeatureLayer
from arcgis.features.analysis import overlay_layers
import sys
sys.path.append('../../')
from utils import get_config

In [2]:
username = get_config("arcgis","username")
password = get_config("arcgis","passkey")
gis = GIS("https://ucsdonline.maps.arcgis.com/home", username=username, password=password)

In [27]:
sd_neighborhood_lyr = FeatureLayer(gis=gis,
                                  url="https://services1.arcgis.com/eGSDp8lpKe5izqVc/arcgis/rest/services/add351/FeatureServer/0")

In [28]:
san_diego_block_group_lyr = FeatureLayer(gis=gis,
                                        url="https://services1.arcgis.com/eGSDp8lpKe5izqVc/arcgis/rest/services/a38898/FeatureServer/0") 

In [29]:
%%time
neigh_sd_block_grp_overlay_features = overlay_layers(input_layer=sd_neighborhood_lyr,
                                              overlay_layer=san_diego_block_group_lyr,
                                              overlay_type='Intersect',
                                              output_name='neigh_sd_block_grp_overlay_features',
                                             )

{"cost": 2.2}


CPU times: user 265 ms, sys: 299 ms, total: 564 ms
Wall time: 55.4 s


In [30]:
neigh_sd_block_grp_overlay_features

<Item title:"neigh_sd_block_grp_overlay_features" type:Feature Layer Collection owner:akale_UCSD>

In [31]:
sdf = neigh_sd_block_grp_overlay_features.layers[0].query().sdf
sdf

,OBJECTID_1,FID_ADD351_ADD351,FID_1,OBJECTID,CPANAME,ShapeSTAre,ShapeSTLen,Shape__Are,Shape__Len,FID_A38898_A38898,state_abbr,state_fips,county_fip,tract_fips,blockgroup,fips,Shape__Area_1,Shape__Length_1,AnalysisArea,SHAPE
0,1,1,1.0,1,"SOUTHEASTERN SAN DIEGO,SOUTHEASTERN",132842908.883943,51202.267742,17477623.855469,18565.577657,78,CA,6,73,2502,1,060730025021,1458636.324219,5591.997008,0.000001,"{""rings"": [[[-13036870.7833, 3857942.2741], [-..."
1,2,1,1.0,1,"SOUTHEASTERN SAN DIEGO,SOUTHEASTERN",132842908.883943,51202.267742,17477623.855469,18565.577657,237,CA,6,73,3301,1,060730033011,644656.578125,3637.437669,0.17577,"{""rings"": [[[-13034955.6507, 3855612.1665], [-..."
2,3,1,1.0,1,"SOUTHEASTERN SAN DIEGO,SOUTHEASTERN",132842908.883943,51202.267742,17477623.855469,18565.577657,238,CA,6,73,3301,2,060730033012,1515108.535156,5009.597577,0.226556,"{""rings"": [[[-13035522.9312, 3856569.6181], [-..."
3,4,1,1.0,1,"SOUTHEASTERN SAN DIEGO,SOUTHEASTERN",132842908.883943,51202.267742,17477623.855469,18565.577657,239,CA,6,73,3303,1,060730033031,230719.578125,2388.726888,0.062851,"{""rings"": [[[-13034896.8739, 3854171.7923], [-..."
4,5,1,1.0,1,"SOUTHEASTERN SAN DIEGO,SOUTHEASTERN",132842908.883943,51202.267742,17477623.855469,18565.577657,240,CA,6,73,3303,2,060730033032,651000.503906,3372.386603,0.177519,"{""rings"": [[[-13034959.2129, 3855128.2743], [-..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2352,2353,89,26.0,26,County Islands,28256.983101,686.797288,3723.460938,249.426485,1066,CA,6,73,14806,1,060730148061,302219.898438,2521.882983,0.0,"{""rings"": [[[-13028824.4372, 3864998.2925], [-..."
2353,2354,58,58.0,58,COLLEGE AREA,85759366.435176,48266.233622,11300530.460938,17501.312153,1067,CA,6,73,14806,2,060730148062,985078.246094,7190.890698,0.0,"{""rings"": [[[-13028863.1193, 3864997.7125], [-..."
2354,2355,89,26.0,26,County Islands,28256.983101,686.797288,3723.460938,249.426485,1067,CA,6,73,14806,2,060730148062,985078.246094,7190.890698,0.0,"{""rings"": [[[-13028863.1193, 3864997.7125], [-..."
2355,2356,101,38.0,38,Spring Valley,325953424.23037,94672.6674,42903411.949219,34346.166675,1012,CA,6,73,13801,1,060730138011,1819231.722656,7069.472438,0.0,"{""rings"": [[[-13024983.0245, 3862507.8976], [-..."


In [32]:
sdf['CPANAME'] = sdf['CPANAME'].str.lower()
sdf[['CPANAME','fips']].nunique()

CPANAME      81
fips       1408
dtype: int64

In [33]:
sdf.shape

(2357, 20)

In [34]:
sdf[['CPANAME','fips']].drop_duplicates()

,CPANAME,fips
0,"southeastern san diego,southeastern",060730025021
1,"southeastern san diego,southeastern",060730033011
2,"southeastern san diego,southeastern",060730033012
3,"southeastern san diego,southeastern",060730033031
4,"southeastern san diego,southeastern",060730033032
...,...,...
2196,north county metro,060730207112
2203,north county metro,060730200382
2204,north county metro,060730200401
2206,county islands,060730033012


In [35]:
# Saving the neighbourhood and block group fip mapping
sdf[['CPANAME','fips']].drop_duplicates().to_csv("../resources/san_diego_county_cpa_blk_grp_map.csv", index=False)